In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [105]:
## some are coming from the notebook: Old trap - 20211001 - All calibration with comments
bandwidth = 2
decimate = 5
drive_freq = 139
fsamp = 5000

scaleZ = 121.96
voltage_calib = 23

E1, E2 = (131, 195)
E_tot = E1+E2
factor = 0.621

In [106]:
i = 0
fname = r'/data/old_trap/20211001/bead1/neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo5/Z_200Vpp_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[1])
voltage_200 = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage_200)

voltage:  190.87201524


In [9]:
etha = 1/scaleZ*E_tot*voltage_calib*1.6e-19 ## convert bits to force
to_epsilon = 1/(2*131*1.6e-19*voltage_200) ## this is 2E1 - the convertion factor from A to epsilon

### Look at neutrality data

In [10]:
def get_amp_wraperZ(i, fname):
    try:
        return get_ampZ_second_harmonic(i, fname)
    except:
        return 0

In [175]:
ll = likelihood_analyser.LikelihoodAnalyser()

def get_ampZ_second_harmonic(i, foldername):
    
    fname = foldername+'Z_200Vpp_'+str(i)+'.h5'
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    drive_freq = 139
    jj = i%2
    freq2 = np.fft.rfftfreq(len(bb.other_data[1+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[1+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

    phase_diff = 4.8        
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-100000, 100000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma':[100,3000000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    phase_diff = 5.7   
    drive_freq = drive_freq*2
    
    fit_kwargs = {'A': 0, 'f': 139*2, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [0,150], 'fix_f': True, 'fix_phi': False,
                  'print_level': 0}

#     fit_kwargs = {'A': 0, 'f': drive_freq*2, 'phi': 0, 'sigma': 1100,
#                   'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
#                   'limit_A': [0, 100000], 'fix_sigma':True, 'fix_phi': False, 'fix_f': True,
#                   'print_level': 0}
    
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    nsamp = 50000
    nsamp = len( bb.pos_data[2])
    tarr = np.arange(0, nsamp/fsamp, 1.0/fsamp)
    sig = np.sin(2.0 * np.pi * drive_freq * tarr+phi_tmp2)

    corr = correlation(sig, bb.pos_data[2] , fsamp=5000, fdrive=139*2, filt=False)[0]

    return corr, m2_tmp.values[0], m1_tmp.values[3], (m2_tmp.values[2]-phi_tmp2)

In [176]:
base_folder = r'/data/old_trap/20211001/bead1/'
fname = base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_93kHz_200Vpp_delay_3sec_with_height_fb/'
get_ampZ_second_harmonic(33, fname)

(92.335601359979037,
 96.17648026910555,
 1639.9315276025404,
 array([-0.39510762]))

In [224]:
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = base_folder+r'neutrality11/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_200Vpp_delay_6sec_with_height_fb/'

# base_folder = r'/data/old_trap/20211001/bead1/'
# fnames = base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_5sec_with_height_fb/'

amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fname) for i in tqdm(range(2000))))


100%|██████████| 2000/2000 [00:31<00:00, 62.07it/s]/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


In [246]:
amp_fit = (amps_tmp[::2,3])%(2*np.pi)
amp_fit1 = (amps_tmp[1::2,3])%(2*np.pi)

_,ax = plt.subplots(2,1,figsize=(9.3,6), sharex=True)
ax[0].plot(range(len(amp_fit)), amp_fit, '+')
ax[0].plot(range(len(amp_fit1)), amp_fit1, '*')

phase_fit = amps_tmp[::2,1]/(1.2e5*200)/1.6e-19*1e6*etha*2
phase_fit1 = amps_tmp[1::2,1]/(1.7e5*200)/1.6e-19*1e6*etha*2

ax[1].scatter(range(len(phase_fit)), phase_fit)
ax[1].scatter(range(len(phase_fit1)), phase_fit1)

ax[1].set(xlabel='file #', ylabel=r'induced edm [$e \, \mu m$]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'file #'), Text(0, 0.5, 'induced edm [$e \\, \\mu m$]')]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'file #'), Text(0, 0.5, 'induced edm [$e \\, \\mu m$]')]

In [225]:
phase_fit = amps_tmp[::2,1]*etha
phase_fit1 = amps_tmp[1::2,1]*etha

_,ax = plt.subplots(figsize=(9.3,4))
ax.scatter(range(len(phase_fit)), phase_fit)
ax.scatter(range(len(phase_fit1)), phase_fit1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [219]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
phase_calib = -0.0563
# phase_calib = -0.0442

def get_amp_new_trap(i):
    
    fname = r'/data/new_trap/20210622/bead1/Neutrality2/Relaxation_20210708/200V_2/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)

    drive_freq = 71
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 'sigma': 0.1,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':0.1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 'limit_sigma': [0, 1],
                  'limit_A': [-100, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.y2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = 142+1
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': np.pi/2, 'sigma': 0.1,
              'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':0.1,
               'fix_phi': False, 'limit_sigma': [0, 1],
              'limit_A': [0, 100], 
              'print_level': 0, 'fix_f': True}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.y2)[::decimate]
    
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0],  m2_tmp.values[0], m2_tmp.values[2]%(2*np.pi)

In [220]:
get_amp_new_trap(0)

(3.3359027753476767e-06, 8.494699066385891e-06, 2.10350868661547)

In [221]:
amps_tmp_new_trap_noise = np.array(Parallel(n_jobs=40)(delayed(get_amp_new_trap)(i) for i in tqdm(range(2000))))


100%|██████████| 2000/2000 [00:27<00:00, 91.74it/s]

In [216]:
etha_new = 1/3.54e-05*(62.9+86.2)*10*1.6e-19 ## convert bits to force

In [245]:
amp_fit = (amps_tmp_new_trap[2::4,2])%(2*np.pi)
amp_fit1 = (amps_tmp_new_trap_noise[2::4,2])%(2*np.pi)

_,ax = plt.subplots(2,1,figsize=(9.3,6), sharex=True)
ax[0].plot(range(len(amp_fit)), amp_fit, '+')
ax[0].plot(range(len(amp_fit1)), amp_fit1, '*')
ax[0].set(ylabel=r'phase [rad]')

phase_fit = amps_tmp_new_trap[2::4,1]/(3.2e4*200)/1.6e-19*1e6*etha_new*2
phase_fit1 = amps_tmp_new_trap_noise[2::4,1]/(3.2e4*200)/1.6e-19*1e6*etha_new*2

ax[1].scatter(range(len(phase_fit)), phase_fit,label='second harmonic - bottom electrode')
ax[1].scatter(range(len(phase_fit1)), phase_fit1,label='sideband')
ax[1].legend()
ax[1].set(xlabel='file #', ylabel=r'induced edm [$e \, \mu m$]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'file #'), Text(0, 0.5, 'induced edm [$e \\, \\mu m$]')]

In [240]:
phase_fit = amps_tmp_new_trap[2::4,1]/(3.2e4*200)/1.6e-19*1e6*etha_new*2
phase_fit1 = amps_tmp_new_trap_noise[2::4,1]/(3.2e4*200)/1.6e-19*1e6*etha_new*2

_,ax = plt.subplots(figsize=(9.3,4))
ax.scatter(range(len(phase_fit)), phase_fit,label='second harmonic - bottom electrode')
ax.scatter(range(len(phase_fit1)), phase_fit1,label='sideband')
ax.legend()
ax.set(xlabel='file #', ylabel=r'induced edm [$e \, \mu m$]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'file #'), Text(0, 0.5, 'induced edm [$e \\, \\mu m$]')]

In [230]:
phase_fit = amps_tmp_new_trap[::4,1]/(2.6e4*200)/1.6e-19*1e6*etha_new*2
# phase_fit1 = amps_tmp_new_trap[1::2,1]/(3.2e4*200)/1.6e-19*1e6*etha_new*2
phase_fit1 = amps_tmp_new_trap_noise[2::4,1]/(2.6e4*200)/1.6e-19*1e6*etha_new*2

_,ax = plt.subplots(figsize=(9.3,4))
ax.scatter(range(len(phase_fit)), phase_fit)
ax.scatter(range(len(phase_fit1)), phase_fit1)
ax.set(xlabel='file #', ylabel=r'induced edm [$e \, \mu m$]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …